# Imports

In [ ]:
import subprocess

In [ ]:
import altair as alt
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
from tqdm import tqdm

In [ ]:
from utils import CHAR_LOOKUP, flatten_columns, get_datadir

In [ ]:
# allow pandas to show more data
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 1000)
alt.data_transformers.enable("vegafusion")
# Create new `pandas` methods which use `tqdm` progress
# (can use tqdm_gui, optional kwargs, etc.)
tqdm.pandas()

# Select year and load file

In [ ]:
year: str = "2003-2004"

In [ ]:
datadir = get_datadir(year)
datadir

In [ ]:
xptfile = datadir / f"paxraw_{CHAR_LOOKUP[year].lower()}.xpt"
zipfilename = f"PAXRAW_{CHAR_LOOKUP[year].upper()}.ZIP"
zipfile = datadir / zipfilename
if not xptfile.exists():
    print("no extracted xpt file, looking for the zip")
    if not zipfile.exists():
        print("no zip exists, downloading it")
        subprocess.run(
            [
                "wget",
                "-O",
                zipfile,
                f"https://wwwn.cdc.gov/Nchs/Nhanes/{year}/{zipfilename}",
                "--no-use-server-timestamps",
            ]
        )
    print("extracting")
    subprocess.run(["unzip", "-o", zipfile, "-d", datadir])
else:
    print("xpt file exists, carry on")

In [ ]:
paxraw = pd.read_sas(xptfile)

In [ ]:
paxraw.shape

In [ ]:
paxraw.head()

## Fix datatypes

In [ ]:
paxraw.dtypes

In [ ]:
for col in paxraw.columns:
    print(f"casting {col=} to int")
    try:
        paxraw.loc[:, col] = paxraw.loc[:, col].astype(int)
    except pd.errors.IntCastingNaNError:
        print(f"{col=} has {paxraw.loc[:, col].isna().sum()} NA values, setting to 0")
        paxraw.loc[:, col] = paxraw.loc[:, col].replace(np.nan, 0).astype(int)

### Don't add a datetime column, takes too long

```
paxraw["datetime"] = paxraw.progress_apply(
    lambda x: datetime.datetime(2006, 1, 1) + datetime.timedelta(
        days=int(x.PAXDAY - 1),
        hours=int(x.PAXHOUR),
        minutes=int(x.PAXMINUT)
    ),
    axis=1,
)
```

In [ ]:
paxraw.dtypes

In [ ]:
paxraw.head()

## Save parquet

In [ ]:
parquetfile = datadir / f"paxraw_{CHAR_LOOKUP[year].lower()}.parquet"
parquetfile

In [ ]:
paxraw.to_parquet(parquetfile)
# paxraw = pd.read_parquet(parquetfile)

# Define intensity level cuts and METs

In [ ]:
# cuts defined in literature and in [common software](https://github.com/vandomed/nhanesaccel/blob/7ebd7a0cd6e2f169e6f81a66c8c99b1746eacb51/R/process_nhanes.R#L267)
int_cuts = [100, 760, 2020, 5999]

In [ ]:
# add end ranges for interpolation
int_cuts_endranges = [paxraw.PAXINTEN.min()] + int_cuts + [paxraw.PAXINTEN.max() + 1]
int_cuts_endranges

In [ ]:
len(int_cuts_endranges) - 1

In [ ]:
# MET values corresponding to each cut point
METs = [1, 1, 2, 3.5, 6, 10]
labels = ["Sedentary", "Low", "Light", "Moderate", "Vigorous"]

In [ ]:
# linearly interpolate MET values
METs_full = np.interp(
    np.arange(int_cuts_endranges[0], int_cuts_endranges[-1]), int_cuts_endranges, METs
)
METs_lookup = pd.DataFrame(
    {
        "MET": METs_full,
        "PAXINTEN": np.arange(int_cuts_endranges[0], int_cuts_endranges[-1]),
    }
)

## Join METs

In [ ]:
paxraw = paxraw.merge(METs_lookup, how="left", on="PAXINTEN")

## Save parquet

In [ ]:
parquetfile = datadir / f"paxraw_{CHAR_LOOKUP[year].lower()}_met.parquet"
parquetfile

In [ ]:
paxraw.to_parquet(parquetfile)

# `worn` classification

[R code here](https://github.com/vandomed/nhanesaccel/blob/7ebd7a0cd6e2f169e6f81a66c8c99b1746eacb51/misc/process_nhanes_app_10_2_18.R)

## Run a sample of data through

In [ ]:
paxraw_sample = paxraw.loc[paxraw.SEQN == paxraw.SEQN.values[0], :].copy()
paxraw_sample.shape

In [ ]:
min_worn_hours_threshold: int = 10
max_nonzero_count_per_unworn_hour: int = 2
max_of_nonzero_in_unworn_hour: int = 100
MINUTES_PER_HOUR = 60

In [ ]:
paxraw_sample.columns

In [ ]:
# set the indicator to True to start
worn = np.ones(paxraw_sample.shape[0])

In [ ]:
worn.shape

paxraw = paxraw_sample

In [ ]:
paxinten = paxraw_sample.PAXINTEN.values

In [ ]:
paxinten.shape[0]

## Time a simple algorithm using numpy arrays

In [ ]:
# take the first hour
# assert d.iloc[:MINUTES_PER_HOUR, :].shape[0] == MINUTES_PER_HOUR
if ((paxinten[:MINUTES_PER_HOUR] > 0).sum() <= max_nonzero_count_per_unworn_hour) and (
    (paxinten[:MINUTES_PER_HOUR] < max_of_nonzero_in_unworn_hour).sum() == MINUTES_PER_HOUR
):
    worn[:MINUTES_PER_HOUR] = 0

In [ ]:
for i in range(MINUTES_PER_HOUR + 1, worn.shape[0]):
    # assert paxraw_sample.iloc[(i-60):i, :].shape[0] == MINUTES_PER_HOUR
    if ((paxinten[(i - MINUTES_PER_HOUR) : i] > 0).sum() <= max_nonzero_count_per_unworn_hour) and (
        (paxinten[(i - MINUTES_PER_HOUR) : i] < max_of_nonzero_in_unworn_hour).sum()
        == MINUTES_PER_HOUR
    ):
        worn[(i - MINUTES_PER_HOUR) : i] = 0

### Write that as a function (in `util.py`)

In [ ]:
from utils import worn_indicator, worn_indicator_fast  # noqa: E402

In [ ]:
%%timeit
worn_indicator(paxraw_sample.PAXINTEN.values)

Run it once to compile it

In [ ]:
worn_indicator_fast(paxraw_sample.PAXINTEN.values)

In [ ]:
%%timeit
worn_indicator_fast(paxraw_sample.PAXINTEN.values)

## Test an algorithm using pandas

### Process out active minutes akin to Fishman (2016)

1. Compute worn/nonworn indicator on each minute, defined as intervals at least 60 minutes of count = 0, with up to two count < 100.
2. Sum worn time per day.
3. Discard days with wear time < 10h.
4. Sum up total count per day.
4. Measure average total count per day on valid days, per individual.

In [ ]:
paxraw_sample.head()

In [ ]:
paxraw_sample.head()

In [ ]:
# set the indicator to True to start
paxraw_sample.loc[:, "worn"] = True

In [ ]:
paxraw_sample.columns

In [ ]:
PAXINTEN_col = np.arange(paxraw_sample.shape[1])[paxraw_sample.columns == "PAXINTEN"][0]

In [ ]:
worn_col = np.arange(paxraw_sample.shape[1])[paxraw_sample.columns == "worn"][0]

In [ ]:
# take the first 60 minutes
assert paxraw_sample.iloc[:60, :].shape[0] == 60
if (paxraw_sample.iloc[:60, PAXINTEN_col] >= 100).sum() <= 2:
    paxraw_sample.iloc[:60, worn_col] = False

In [ ]:
paxraw_sample.iloc[paxraw_sample.shape[0] - 60 : paxraw_sample.shape[0], :].shape

In [ ]:
for i in range(61, paxraw_sample.shape[0]):
    assert paxraw_sample.iloc[(i - 60) : i, :].shape[0] == 60
    if (paxraw_sample.iloc[(i - 60) : i, PAXINTEN_col] >= 100).sum() <= 2:
        paxraw_sample.iloc[(i - 60) : i, worn_col] = False

In [ ]:
worn_chart = (
    alt.Chart(paxraw_sample)
    .mark_bar(width=1)
    .encode(x="PAXN:O", y=alt.value(-10), y2=alt.value(2), color="worn")
)
inten_chart = alt.Chart(paxraw_sample).mark_line(color="orange").encode(x="PAXN:O", y="PAXINTEN")
if "PAXSTEP" in paxraw_sample.columns:
    step_chart = alt.Chart(paxraw_sample).mark_line().encode(x="PAXN:O", y="PAXSTEP")
    chart = worn_chart + inten_chart + step_chart
else:
    chart = worn_chart + inten_chart
chart.properties(width=1400, height=600)

In [ ]:
# set the indicator to True to start
paxraw_sample.loc[:, "worn"] = True
# take the first 60 minutes
assert paxraw_sample.iloc[:60, :].shape[0] == 60
# only 2 allowed > 0, and all 60 are less than 100
if ((paxraw_sample.iloc[:60, PAXINTEN_col] > 0).sum() <= 2) and (
    (paxraw_sample.iloc[:60, PAXINTEN_col] < 100).sum() == 60
):
    paxraw_sample.iloc[:60, worn_col] = False

In [ ]:
for i in range(61, paxraw_sample.shape[0]):
    assert paxraw_sample.iloc[(i - 60) : i, :].shape[0] == 60
    if ((paxraw_sample.iloc[(i - 60) : i, PAXINTEN_col] > 0).sum() <= 2) and (
        (paxraw_sample.iloc[(i - 60) : i, PAXINTEN_col] < 100).sum() == 60
    ):
        paxraw_sample.iloc[(i - 60) : i, worn_col] = False

In [ ]:
(
    alt.Chart(paxraw_sample)
    .mark_bar(width=5, opacity=0.3)
    .encode(
        x="PAXN:O",
        y=alt.value(600),
        y2=alt.value(2),
        color=alt.Color("worn", scale=alt.Scale(range=["white", "grey"])),
    )
    + alt.Chart(paxraw_sample)
    .mark_line(color="orange", clip=True)
    .encode(x="PAXN:O", y=alt.Y("PAXINTEN", scale=alt.Scale(domain=[0, 5000])))
).properties(width=1400, height=600)

In [ ]:
# set the indicator to False to start
paxraw_sample.loc[:, "worn"] = False
# take the first 60 minutes
assert paxraw_sample.iloc[:60, :].shape[0] == 60
# only 2 allowed > 0, and all 60 are less than 100
if ((paxraw_sample.iloc[:60, PAXINTEN_col] > 0).sum() > 2) or (
    (paxraw_sample.iloc[:60, PAXINTEN_col] >= 100).sum() > 0
):
    paxraw_sample.iloc[:60, worn_col] = True

In [ ]:
for i in range(61, paxraw_sample.shape[0]):
    assert paxraw_sample.iloc[(i - 60) : i, :].shape[0] == 60
    if ((paxraw_sample.iloc[(i - 60) : i, PAXINTEN_col] > 0).sum() > 2) or (
        (paxraw_sample.iloc[(i - 60) : i, PAXINTEN_col] >= 100).sum() > 0
    ):
        paxraw_sample.iloc[(i - worn_col) : i, worn_col] = True

In [ ]:
(
    alt.Chart(paxraw_sample)
    .mark_bar(width=5, opacity=0.3)
    .encode(
        x="PAXN:O",
        y=alt.value(600),
        y2=alt.value(2),
        color=alt.Color("worn", scale=alt.Scale(range=["white", "grey"])),
    )
    + alt.Chart(paxraw_sample)
    .mark_line(color="orange", clip=True)
    .encode(x="PAXN:O", y=alt.Y("PAXINTEN", scale=alt.Scale(domain=[0, 5000])))
).properties(width=1400, height=600)

In [ ]:
worn_minutes = paxraw_sample.groupby("PAXDAY").agg({"worn": [sum], "PAXINTEN": [sum]})
worn_minutes["valid_day"] = worn_minutes["worn"]["sum"] > min_worn_hours_threshold * 60
# filter to valid days
worn_minutes = worn_minutes.loc[worn_minutes.valid_day, :]
np.mean(worn_minutes["PAXINTEN"]["sum"])

In [ ]:
from utils import get_person_active_count  # noqa: E402

In [ ]:
get_person_active_count(paxraw.loc[paxraw.SEQN == paxraw.SEQN.unique()[0], :])

In [ ]:
%%timeit
get_person_active_count(paxraw.loc[paxraw.SEQN == paxraw.SEQN.unique()[0], :])

Hours:

In [ ]:
3.13 * (paxraw["SEQN"].unique().shape[0]) / 60 / 60

### Test it on a slightly bigger sample

Make sure the groupby object returned makes sense before waiting 8 hours

In [ ]:
person_active_counts = (
    paxraw.loc[paxraw.SEQN.isin(paxraw.SEQN.unique()[:10]), :]
    .groupby("SEQN")
    .progress_apply(get_person_active_count)
)

In [ ]:
person_active_counts

In [ ]:
# check that we can save and load it
# with the heirarchical indexes
parquetfile = datadir / f"paxraw_{CHAR_LOOKUP[year].lower()}_sample_test_write.parquet"
person_active_counts.to_parquet(parquetfile)
pd.read_parquet(parquetfile).head()

### Don't apply the numpy-based function to Pandas column, too slow

Because it takes almost 30 minutes.

this would take ~25 minutes
```
paxraw['worn'] = 1

for SEQN in tqdm(pd.unique(paxraw.SEQN.values)):
    paxraw.loc[paxraw.SEQN == SEQN, 'worn'] = worn_indicator(paxraw.loc[paxraw.SEQN == SEQN, 'PAXINTEN'].values)
```

```
parquetfile = datadir / f"paxraw_{CHAR_LOOKUP[year].lower()}_met_worn.parquet"
paxraw.to_parquet(parquetfile)
```

### Skip a fully pandas-based solution entirely, it's very very slow

FWIW, the `rolling` version should take better advantage of Pandas,
but it's still too slow.

```
person_active_counts = (
    paxraw.groupby("SEQN").progress_apply(get_person_active_count).reset_index()
)
```

## Apply ~numpy-based~ numba algorithm to full dataset

~Less than 10 min.~

A few seconds.

In [ ]:
from utils import bout_classifier_SEQN_long, worn_indicator_SEQN_long_fast  # noqa: E402

In [ ]:
# this should be fast
paxraw["worn"] = worn_indicator_SEQN_long_fast(paxraw.PAXINTEN.values, paxraw.SEQN.values)

### Compare the full numpy and the numpy function applied to pandas array

It's commented out because we're not running the "numpy function applied to pandas array" version now.
They are the same

```
(paxraw['worn'] == worn).sum()
```

```
(paxraw['worn'] != worn).sum()
```

```
(paxraw['worn'] == worn).sum() == worn.shape[0]
```

```
paxraw.loc[(paxraw.worn != worn), :].head()
```

## Save parquet

In [ ]:
parquetfile = datadir / f"paxraw_{CHAR_LOOKUP[year].lower()}_met_worn.parquet"
parquetfile

In [ ]:
paxraw.to_parquet(parquetfile)

# Generate indicators for `bouts` of activity levels

In [ ]:
paxraw["vigorous_bout"] = bout_classifier_SEQN_long(
    paxraw.PAXINTEN.values,
    paxraw.SEQN.values,
    paxraw.worn.values,
    np.zeros(paxraw.PAXINTEN.values.shape[0]),
    upper=int_cuts_endranges[5],
    lower=int_cuts_endranges[4],
    tol_upper_soft=0,
    tol_lower_soft=0,
    m=10,
    lower_soft=(int_cuts_endranges[3] + int_cuts_endranges[4]) / 2,
    check_already_classified=False,
)

In [ ]:
paxraw["moderate_bout"] = bout_classifier_SEQN_long(
    paxraw.PAXINTEN.values,
    paxraw.SEQN.values,
    paxraw.worn.values,
    paxraw.vigorous_bout.values,
    upper=int_cuts_endranges[4],
    lower=int_cuts_endranges[3],
    tol_upper_soft=10,
    tol_lower_soft=0,
    m=10,
    lower_soft=(int_cuts_endranges[2] + int_cuts_endranges[3]) / 2,
    upper_soft=int_cuts_endranges[5],
    check_already_classified=True,
)

In [ ]:
paxraw["light_bout"] = bout_classifier_SEQN_long(
    paxraw.PAXINTEN.values,
    paxraw.SEQN.values,
    paxraw.worn.values,
    np.maximum(paxraw.moderate_bout.values, paxraw.vigorous_bout.values),
    upper=int_cuts_endranges[3],
    lower=int_cuts_endranges[2],
    tol_upper_soft=10,
    tol_lower_soft=0,
    m=10,
    lower_soft=(int_cuts_endranges[1] + int_cuts_endranges[2]) / 2,
    upper_soft=int_cuts_endranges[5],
    check_already_classified=True,
)

In [ ]:
paxraw["low_bout"] = bout_classifier_SEQN_long(
    paxraw.PAXINTEN.values,
    paxraw.SEQN.values,
    paxraw.worn.values,
    np.maximum(paxraw.light_bout.values, paxraw.moderate_bout.values, paxraw.vigorous_bout.values),
    upper=int_cuts_endranges[2],
    lower=int_cuts_endranges[1],
    tol_upper_soft=10,
    tol_lower_soft=0,
    m=10,
    lower_soft=(int_cuts_endranges[0] + int_cuts_endranges[1]) / 2,
    upper_soft=int_cuts_endranges[5],
    check_already_classified=True,
)

In [ ]:
paxraw["sed_bout"] = bout_classifier_SEQN_long(
    paxraw.PAXINTEN.values,
    paxraw.SEQN.values,
    paxraw.worn.values,
    paxraw.low_bout.values,
    upper=int_cuts_endranges[1],
    lower=int_cuts_endranges[0],
    tol_upper_soft=0,
    tol_lower_soft=0,
    m=10,
    check_already_classified=False,
)

### Add them all to the dataframe

In [ ]:
paxraw["no_bout"] = (
    (paxraw["worn"] == 1)
    & (paxraw["sed_bout"] == 0)
    & (paxraw["low_bout"] == 0)
    & (paxraw["light_bout"] == 0)
    & (paxraw["moderate_bout"] == 0)
    & (paxraw["vigorous_bout"] == 0)
) * 1

## Save parquet

In [ ]:
parquetfile = datadir / f"paxraw_{CHAR_LOOKUP[year].lower()}_met_worn_bouts.parquet"
parquetfile

In [ ]:
paxraw.to_parquet(parquetfile)
# paxraw = pd.read_parquet(parquetfile)

## A single column to label minute-by-minute intensity

In [ ]:
paxraw["intensity"] = pd.cut(
    paxraw.PAXINTEN.values, int_cuts_endranges, right=False, labels=range(len(labels))
)
# don't include the labels for size:
# labels=labels

In [ ]:
paxraw.head()

In [ ]:
paxraw["METh"] = paxraw.MET / 60
paxraw["activeMETh"] = (paxraw.MET - 1) / 60

In [ ]:
paxraw_sample = paxraw.loc[paxraw.SEQN == paxraw.SEQN.values[0], :].copy()
paxraw_sample.head()

In [ ]:
(
    alt.Chart(paxraw_sample)
    .mark_bar(width=1)
    .encode(
        x="PAXN:O",
        y=alt.value(400),
        y2=alt.value(2),
        color=alt.Color("intensity", scale=alt.Scale(scheme="orangered"))
        # scale=alt.Scale(range=["white", "grey"])),
    )
    + alt.Chart(paxraw_sample)
    .mark_line(color="#1f77b4", clip=True)
    .encode(x="PAXN:O", y=alt.Y("PAXINTEN", scale=alt.Scale(domain=[0, 8000])))
).properties(width=1400, height=400)

In [ ]:
(
    alt.Chart(paxraw_sample)
    .mark_bar()
    .encode(
        x="PAXDAY:O",
        y=alt.Y("count()", title="Minutes"),
        color=alt.Color("intensity", scale=alt.Scale(scheme="orangered"))
        # scale=alt.Scale(range=["white", "grey"])),
    )
).properties(width=400, height=400)

In [ ]:
(
    alt.Chart(paxraw_sample)
    .mark_bar()
    .encode(
        x="PAXDAY:O",
        y=alt.Y("sum(METh)", title="MET-h"),
        color=alt.Color("intensity", scale=alt.Scale(scheme="orangered"))
        # scale=alt.Scale(range=["white", "grey"])),
    )
).properties(width=400, height=400)

In [ ]:
(
    alt.Chart(paxraw_sample)
    .mark_bar()
    .encode(
        x="PAXDAY:O",
        y=alt.Y("sum(activeMETh)", title="MET-h"),
        color=alt.Color("intensity", scale=alt.Scale(scheme="orangered"))
        # scale=alt.Scale(range=["white", "grey"])),
    )
).properties(width=400, height=400)

## Check for overlap on intensity bounts

First sum it up

In [ ]:
paxraw.loc[
    :,
    [
        "worn",
        "sed_bout",
        "low_bout",
        "light_bout",
        "moderate_bout",
        "vigorous_bout",
        "no_bout",
    ],
].sum(axis=0)

From before the generalized numba function:
```
"worn             34326726.0\n",
"sed_bout         10920764.0\n",
"low_bout           604376.0\n",
"light_bout          62892.0\n",
"moderate_bout       56051.0\n",
"vigorous_bout      395787.0\n",
"no_bout          22352901.0\n",
```

In [ ]:
((paxraw["worn"] == 1) & (paxraw["sed_bout"] == 0)).sum()

In [ ]:
((paxraw["worn"] == 1) & (paxraw["low_bout"] == 1)).sum()

In [ ]:
((paxraw["sed_bout"] == 1) & (paxraw["low_bout"] == 1)).sum()

In [ ]:
((paxraw["sed_bout"] == 1) & (paxraw["light_bout"] == 1)).sum()

In [ ]:
((paxraw["sed_bout"] == 1) & (paxraw["moderate_bout"] == 1)).sum()

In [ ]:
((paxraw["sed_bout"] == 1) & (paxraw["vigorous_bout"] == 1)).sum()

# Get valid days and other filters

## Worn minutes by person-day to compute `valid_day`

In [ ]:
# sum minutes of wear and activity counts per day
worn_minutes = paxraw.groupby(["SEQN", "PAXDAY"]).agg({"worn": [np.sum]})

worn_minutes.columns = flatten_columns(worn_minutes.columns.values)

In [ ]:
# compute valid days
worn_minutes["valid_day"] = (
    worn_minutes["worn"]["sum"] > (min_worn_hours_threshold * MINUTES_PER_HOUR)
) * 1

In [ ]:
worn_minutes.head(15)

In [ ]:
worn_minutes.columns = flatten_columns(worn_minutes.columns.values)

In [ ]:
worn_minutes.head(15)

## Other indicators at person-day level that can be used to filter

In [ ]:
agg_columns = ["PAXINTEN", "max_intensity", "out_of_calibration", "unreliable"]

In [ ]:
paxraw["max_intensity"] = (paxraw.PAXINTEN == 32767) * 1
paxraw["out_of_calibration"] = (paxraw.PAXCAL == 2) * 1
paxraw["unreliable"] = (paxraw.PAXSTAT == 2) * 1

In [ ]:
if "PAXSTEP" not in paxraw.columns:
    paxraw["PAXSTEP"] = 0
    paxraw["zero_steps_with_intensity"] = 0
else:
    paxraw["zero_steps_with_intensity"] = ((paxraw.PAXINTEN > 250) & (paxraw.PAXSTEP == 0)) * 1

In [ ]:
paxraw["too_many_steps"] = (paxraw.PAXSTEP > 200) * 1

In [ ]:
# add a variable for steps_filtered, summing steps only if we have intensity over 500
paxraw["steps_filtered_500"] = 0
paxraw.loc[paxraw.PAXINTEN >= 500, "steps_filtered_500"] = paxraw.PAXSTEP
paxraw["steps_filtered_300"] = 0
paxraw.loc[paxraw.PAXINTEN >= 300, "steps_filtered_300"] = paxraw.PAXSTEP

In [ ]:
agg_columns += [
    "zero_steps_with_intensity",
    "too_many_steps",
    "steps_filtered_500",
    "steps_filtered_300",
]

In [ ]:
tudor2009_filters = (
    paxraw.groupby(["SEQN", "PAXDAY"])
    .agg({col: [np.sum, "last"] for col in agg_columns})
    .reset_index()
)
tudor2009_filters.columns = flatten_columns(tudor2009_filters.columns.values)

In [ ]:
tudor2009_filters.head(15)

### Join all person-day level indicators

In [ ]:
d_people_days = tudor2009_filters.merge(worn_minutes, how="inner", on=["SEQN", "PAXDAY"])

In [ ]:
d_people_days.head(15)

In [ ]:
parquetfile = datadir / f"paxraw_{CHAR_LOOKUP[year].lower()}_people_days.parquet"
parquetfile

In [ ]:
d_people_days.to_parquet(parquetfile)

## Sum up to person level

In [ ]:
d_people = d_people_days.groupby("SEQN").agg(
    {
        "zero_steps_with_intensity_sum": np.sum,
        "too_many_steps_sum": np.sum,
        "max_intensity_sum": np.sum,
        "out_of_calibration_sum": np.sum,
        "out_of_calibration_last": "last",
        "unreliable_sum": np.sum,
        "unreliable_last": "last",
        "steps_filtered_500_sum": np.mean,
        "steps_filtered_300_sum": np.mean,
        "valid_day": np.sum,
        "PAXINTEN_sum": np.mean,
    }
)

In [ ]:
d_people.head(15)

In [ ]:
d_people.shape

In [ ]:
parquetfile = datadir / f"paxraw_{CHAR_LOOKUP[year].lower()}_people.parquet"
parquetfile

In [ ]:
d_people.to_parquet(parquetfile)

## Use the indicators to filter to people with reliable data

In [ ]:
d_reliable = d_people.loc[
    (d_people.zero_steps_with_intensity_sum <= 10)
    & (d_people.too_many_steps_sum <= 10)
    & (d_people.max_intensity_sum <= 10)
    & (~d_people.out_of_calibration_last)
    & (d_people.unreliable_sum <= 10)
    & (d_people.steps_filtered_500_sum <= 200000),
    :,
]
d_reliable.head(15)

In [ ]:
d_reliable.shape

In [ ]:
parquetfile = datadir / f"paxraw_{CHAR_LOOKUP[year].lower()}_reliable_people.parquet"
parquetfile

In [ ]:
d_reliable.to_parquet(parquetfile)

## Use filtered people to select rows from full data

In [ ]:
paxraw_reliable = paxraw.merge(
    worn_minutes.loc[worn_minutes.valid_day == 1, :], on=["SEQN", "PAXDAY"]
).merge(d_reliable.loc[:, []], how="inner", on="SEQN")
paxraw_reliable.head(10)

In [ ]:
paxraw_reliable.shape

In [ ]:
paxraw_reliable.shape

## Save parquet

In [ ]:
parquetfile = datadir / f"paxraw_{CHAR_LOOKUP[year].lower()}_met_worn_bouts_reliable.parquet"
parquetfile

In [ ]:
paxraw_reliable.to_parquet(parquetfile)

# Look at intensity distribution and METh thresholds

### Group by intensity to sum MET-h levels across days

In [ ]:
groupedMETh = (
    paxraw_sample.groupby(["intensity", "PAXDAY"])
    .agg({"activeMETh": np.sum})
    .groupby(["intensity"])
    .agg({"activeMETh": np.mean})
)
groupedMETh

In [ ]:
groupedMETh.sum()

This is the same as just taking the mean of the sum (without grouping by intensity in the middle):

In [ ]:
paxraw_sample.groupby(["PAXDAY"]).agg({"activeMETh": np.sum}).mean()

In [ ]:
paxraw_reliable.groupby(["SEQN", "intensity", "PAXDAY"]).agg({"activeMETh": np.sum}).head()

In [ ]:
paxraw_reliable.groupby(["SEQN", "intensity", "PAXDAY"]).agg({"activeMETh": np.sum}).groupby(
    ["SEQN", "intensity"]
).agg({"activeMETh": np.mean}).head()

In [ ]:
minutes_at_intensity = (
    paxraw_reliable.groupby(["SEQN", "intensity", "PAXDAY"])
    .agg({"activeMETh": "count"})
    .groupby(["SEQN", "intensity"])
    .agg({"activeMETh": np.mean})
    .groupby(["intensity"])
    .agg({"activeMETh": np.mean})
)
minutes_at_intensity

In [ ]:
METh_at_intensity = (
    paxraw_reliable.groupby(["SEQN", "intensity", "PAXDAY"])
    .agg({"activeMETh": np.sum})
    .groupby(["SEQN", "intensity"])
    .agg({"activeMETh": np.mean})
    .groupby(["intensity"])
    .agg({"activeMETh": np.mean})
)
METh_at_intensity

In [ ]:
minutes_METh = minutes_at_intensity.rename(columns={"activeMETh": "minutes"}).merge(
    METh_at_intensity, how="inner", on="intensity"
)
minutes_METh

In [ ]:
minutes_METh_stack = (
    pd.concat(
        [
            minutes_at_intensity.assign(metric="minutes"),
            METh_at_intensity.assign(metric="MET", activeMETh=lambda d: d.activeMETh * 60),
        ]
    )
    .reset_index()
    .merge(
        pd.DataFrame({"label": labels, "intensity": range(5)}),
        how="left",
        on="intensity",
    )
)
minutes_METh_stack

### MET Minutes vs Minutes by intensity level

In [ ]:
alt.Chart(minutes_METh_stack).mark_bar().encode(
    x=alt.X("metric:N", title="Metric"),
    y=alt.Y("activeMETh:Q", title="Minutes"),
    color=alt.Color("label:O", title="Intensity"),
    column=alt.Column(
        "label:O", title="Intensity", sort=alt.SortField("intensity", order="ascending")
    ),
)

Skip sedentary - no METs

In [ ]:
alt.Chart(minutes_METh_stack.loc[minutes_METh_stack.intensity > 0, :]).mark_bar().encode(
    x=alt.X("metric:N", title="Metric"),
    y=alt.Y("activeMETh:Q", title="Minutes"),
    color=alt.Color("label:O", title="Intensity"),
    column=alt.Column(
        "label:O", title="Intensity", sort=alt.SortField("intensity", order="ascending")
    ),
).properties(width=100, height=400)

## Distribution of weekly METh

In [ ]:
(
    paxraw_reliable.groupby(["SEQN", "intensity", "PAXDAY"])
    .agg({"activeMETh": np.sum})
    .groupby(["SEQN", "intensity"])
    .agg({"activeMETh": np.mean})
    .groupby(["SEQN"])
    .agg({"activeMETh": np.sum})
).mean() * 7

In [ ]:
METh = (
    paxraw_reliable.groupby(["SEQN", "intensity", "PAXDAY"])
    .agg({"activeMETh": np.sum})
    .groupby(["SEQN", "intensity"])
    .agg({"activeMETh": np.mean})
    .groupby(["SEQN"])
    .agg({"activeMETh": np.sum})
).assign(weeklyMETh=lambda d: d.activeMETh * 7)
METh

In [ ]:
alt.Chart(METh).mark_bar().encode(
    alt.X("weeklyMETh:Q", bin=alt.BinParams(maxbins=35)),
    y="count()",
)

## Look at different METh thresholds and % of people getting rewards (and total utilization)

In [ ]:
cut, cuts = pd.qcut(METh.weeklyMETh, 10, retbins=True)
cut

In [ ]:
cuts

In [ ]:
cut.cat.categories

In [ ]:
utilization = pd.DataFrame(
    {
        "target": np.linspace(cuts[1], cuts[-2], num=50),
        "Total Utilization": [
            np.minimum(METh.weeklyMETh.values / x, 1).mean()
            for x in np.linspace(cuts[1], cuts[-2], num=50)
        ],
        "Max Rewards": [
            (METh.weeklyMETh.values / x >= 1).sum() / METh.shape[0]
            for x in np.linspace(cuts[1], cuts[-2], num=50)
        ],
    }
)

In [ ]:
utilization.head()

In [ ]:
alt.Chart(utilization).mark_line().transform_fold(
    fold=["Total Utilization", "Max Rewards"], as_=["variable", "value"]
).encode(
    alt.X("target:Q", title="MET-h for 1000 Points"),
    alt.Y("value:Q", title="Fraction", scale=alt.Scale(zero=False)),
    alt.Color("variable:N", title="Usage"),
).properties(
    width=600
)

In [ ]:
alt.Chart(
    pd.concat(
        [
            pd.DataFrame({"points": np.minimum(METh.weeklyMETh.values / x, 1) * 1000, "MET-h": x})
            for x in np.arange(5, 55, 5)
        ]
    )
).mark_bar().encode(
    alt.Color("points:Q", bin=alt.BinParams(maxbins=20)),
    alt.Y("count()"),
    alt.X("MET-h:O"),
).properties(
    width=600
)

In [ ]:
alt.Chart(
    pd.concat(
        [
            pd.DataFrame({"points": np.minimum(METh.weeklyMETh.values / x, 1) * 1000, "MET-h": x})
            for x in np.arange(5, 55, 5)
        ]
    )
).mark_bar().encode(
    alt.Color(
        "points:Q",
        bin=alt.BinParams(maxbins=20),
        scale=alt.Scale(scheme="redyellowgreen"),
    ),
    alt.Y("count()"),
    alt.X("MET-h:O"),
).properties(
    width=600
)

## Ena's chart of points by intensity for different amounts of points earned

In [ ]:
all_intensities = pd.DataFrame(
    {
        "SEQN": np.repeat(pd.unique(paxraw_reliable.SEQN.values), 5),
        "intensity": np.tile(np.arange(5), pd.unique(paxraw_reliable.SEQN.values).shape[0]),
    }
)
all_intensities.head(11)

In [ ]:
points_by_intensity = (
    paxraw_reliable.groupby(["SEQN", "intensity", "PAXDAY"], dropna=False)
    .agg({"activeMETh": np.sum, "worn": np.sum})
    .groupby(["SEQN", "intensity"], dropna=False)
    .agg({"activeMETh": np.mean, "worn": np.mean})
    # fill in blank intensities
    .merge(all_intensities, how="outer", on=list(all_intensities.columns))
    .fillna(0)
    # calc points
    .assign(weeklyMETh=lambda d: d.activeMETh * 7, points=lambda d: d.weeklyMETh / 25 * 1000)
    .rename(columns={"worn": "dailyMinutes"})
    .reset_index()
)
points_by_intensity.head()

In [ ]:
points_by_person = (
    points_by_intensity.groupby("SEQN")
    .agg({"points": np.sum})
    .assign(
        points_capped=lambda d: np.minimum(d.points, 1000),
        point_bin=lambda d: pd.cut(d.points_capped, np.arange(11) * 100, right=True),
    )
)
points_by_person

In [ ]:
point_thresholds_by_intensity = (
    points_by_intensity.merge(points_by_person, how="left", on="SEQN")
    .assign(
        points_relative=lambda d: d.points_x / d.points_y,
        dailyMinutesCapped=lambda d: d.dailyMinutes * d.points_capped / d.points_y,
    )
    .groupby(["point_bin", "intensity"], dropna=False)
    .agg(
        {
            "points_relative": np.mean,
            "points_capped": np.mean,
            "dailyMinutesCapped": np.mean,
        }
    )
    .assign(
        points=lambda d: d.points_relative * d.points_capped,
    )
    .reset_index()
    .assign(point_bin=lambda d: d.point_bin.astype("str"))
    .merge(pd.DataFrame({"label": labels, "intensity": np.arange(5)}), how="left", on="intensity")
)
point_thresholds_by_intensity

In [ ]:
alt.Chart(
    point_thresholds_by_intensity.loc[point_thresholds_by_intensity.intensity > 0, :]
).mark_bar().encode(
    alt.X("point_bin:O"),
    alt.Y("points:Q", title="Points"),
    alt.Color("label:O", title="Intensity", sort=alt.SortField("label", order="ascending")),
).properties(
    width=500
)

### Stack the bar chart

In [ ]:
alt.Chart(
    point_thresholds_by_intensity.loc[point_thresholds_by_intensity.intensity > 0, :]
).mark_bar().encode(
    alt.X("point_bin:O"),
    alt.Y("points:Q", title="Points", stack="normalize"),
    alt.Color("label:O", title="Intensity", sort=alt.SortField("label", order="ascending")),
).properties(
    width=500
)

### Convert this to daily times in zones

In [ ]:
alt.Chart(
    point_thresholds_by_intensity.loc[point_thresholds_by_intensity.intensity > 0, :]
).mark_bar().encode(
    alt.X("point_bin:O"),
    alt.Y("dailyMinutesCapped:Q", title="Minutes"),
    alt.Detail("label:O", title="Intensity"),
    alt.Color("label:O", title="Intensity", sort=alt.SortField("label", order="ascending")),
    tooltip=["label", "dailyMinutesCapped", "point_bin", "points"],
).properties(
    width=500
)

In [ ]:
alt.Chart(
    point_thresholds_by_intensity.loc[point_thresholds_by_intensity.intensity > 0, :]
).mark_bar().encode(
    alt.X("point_bin:O"),
    alt.Y("dailyMinutesCapped:Q", title="% of time in zone", stack="normalize"),
    alt.Detail("label:O", title="Intensity"),
    alt.Color("label:O", title="Intensity", sort=alt.SortField("label", order="ascending")),
    tooltip=["label", "dailyMinutesCapped", "point_bin", "points"],
).properties(
    width=500
)

In [ ]:
point_thresholds_by_intensity.loc[point_thresholds_by_intensity.intensity > 0, :].pivot(
    index="point_bin", columns="label", values="dailyMinutesCapped"
)

In [ ]:
point_thresholds_by_intensity.loc[point_thresholds_by_intensity.intensity > 0, :].pivot(
    index="point_bin", columns="label", values="dailyMinutesCapped"
).assign(Moderate_and_Vigorous=lambda d: d.Moderate + d.Vigorous * 2).rename(
    columns={"Moderate_and_Vigorous": "Moderate_and_Vigorous".replace("_", " ")}
).loc[
    :, ["Low", "Light", "Moderate and Vigorous"]
].astype(
    "int"
)

In [ ]:
point_thresholds_by_intensity.loc[point_thresholds_by_intensity.intensity > 0, :].pivot(
    index="point_bin", columns="label", values="dailyMinutesCapped"
).assign(
    Moderate_and_Vigorous=lambda d: d.Moderate + d.Vigorous * 2,
    Light=lambda d: d.Low * 0.5 + d.Light,
).rename(
    columns={"Moderate_and_Vigorous": "Moderate_and_Vigorous".replace("_", " ")}
).loc[
    :, ["Light", "Moderate and Vigorous"]
].astype(
    "int"
)

# Additional plots

## Aggregate to daily

In [ ]:
by_day = (
    paxraw.groupby(["SEQN", "PAXDAY"])
    .agg({"PAXSTEP": [sum], "PAXINTEN": [sum, np.mean, max]})
    .reset_index()
)
by_day.head()

In [ ]:
by_day.shape

In [ ]:
# by_day.columns = by_day.columns.get_level_values(0)
by_day.columns = flatten_columns(by_day.columns)

In [ ]:
by_day.loc[by_day.SEQN == 31128.0, :]

## Daily charts

In [ ]:
id = 2
alt.Chart(by_day.loc[by_day.SEQN == by_day.SEQN.unique()[id], :]).mark_bar().encode(
    x="PAXDAY:O", y="PAXSTEP_sum"
).properties(title=f"Steps for SEQN {by_day.SEQN.unique()[id]}")

In [ ]:
alt.Chart(by_day.loc[by_day.SEQN.isin(by_day.SEQN.unique()[:9]), :]).mark_bar().encode(
    x="PAXDAY:O", y="PAXSTEP_sum", column="SEQN:N"
).properties()

## Individual activity/steps

In [ ]:
alt.Chart(paxraw.loc[paxraw.SEQN == by_day.SEQN.unique()[0], :]).mark_line().encode(
    x="PAXN:O", y="PAXSTEP"
).properties(width=1400)

In [ ]:
(
    alt.Chart(paxraw.loc[paxraw.SEQN == by_day.SEQN.unique()[0], :])
    .mark_line()
    .encode(x="PAXN:O", y="PAXSTEP")
    + alt.Chart(paxraw.loc[paxraw.SEQN == by_day.SEQN.unique()[0], :])
    .mark_line(color="orange")
    .encode(x="PAXN:O", y="PAXINTEN")
).properties(width=1400).resolve_scale(y="independent")

In [ ]:
(
    alt.Chart(paxraw.loc[paxraw.SEQN == by_day.SEQN.unique()[0], :])
    .mark_circle(opacity=0.7, color="#F5F5DC", size=120)
    .encode(
        alt.X("PAXINTEN:Q", title="Intensity per minute"),
        alt.Y("PAXSTEP:Q", title="Steps per minute"),
    )
    + alt.Chart(paxraw.loc[paxraw.SEQN == by_day.SEQN.unique()[0], :])
    .mark_circle(opacity=1, color="black", size=3)
    .encode(x="PAXINTEN", y="PAXSTEP")
).properties(width=600, height=600)

In [ ]:
(
    alt.Chart(paxraw.loc[paxraw.SEQN == by_day.SEQN.unique()[0], :])
    .mark_circle(clip=True, opacity=0.7, color="#F5F5DC", size=120)
    .encode(
        alt.X(
            "PAXINTEN:Q",
            title="Intensity per minute",
            scale=alt.Scale(domain=[0, 2500]),
        ),
        alt.Y("PAXSTEP:Q", title="Steps per minute", scale=alt.Scale(domain=[0, 60])),
    )
    + alt.Chart(paxraw.loc[paxraw.SEQN == by_day.SEQN.unique()[0], :])
    .mark_circle(clip=True, opacity=1, color="black", size=3)
    .encode(x="PAXINTEN", y="PAXSTEP")
).properties(width=600, height=600)

In [ ]:
alt.Chart(paxraw.loc[paxraw.SEQN == by_day.SEQN.unique()[0], :]).mark_rect(clip=True).encode(
    alt.X(
        "PAXINTEN:Q",
        title="Intensity per minute",
        scale=alt.Scale(domain=[0, 2500]),
        bin=alt.Bin(maxbins=100),
    ),
    alt.Y(
        "PAXSTEP:Q",
        title="Steps per minute",
        scale=alt.Scale(domain=[0, 60]),
        bin=alt.Bin(maxbins=100),
    ),
    alt.Color("count():Q", scale=alt.Scale(scheme="greenblue", type="log")),
).properties(width=600, height=600)

In [ ]:
alt.Chart(
    paxraw.loc[
        (paxraw.SEQN == by_day.SEQN.unique()[0]) & (paxraw.PAXINTEN > 0) & (paxraw.PAXSTEP > 0),
        :,
    ]
).mark_rect(clip=True).encode(
    alt.X(
        "PAXINTEN:Q",
        title="Intensity per minute",
        scale=alt.Scale(domain=[0, 2500]),
        bin=alt.Bin(maxbins=100),
    ),
    alt.Y(
        "PAXSTEP:Q",
        title="Steps per minute",
        scale=alt.Scale(domain=[0, 60]),
        bin=alt.Bin(maxbins=100),
    ),
    alt.Color("count():Q", scale=alt.Scale(scheme="greenblue", type="log")),
).properties(
    width=600, height=600
)

## Activity plots for a single participant

In [ ]:
person_active_counts = d_people_days.loc[
    :, ["SEQN", "worn_sum", "PAXINTEN_sum", "valid_day"]
].copy()

In [ ]:
person_active_counts.columns = ["SEQN", "worn_minutes", "activity_counts", "valid_day"]

In [ ]:
person_active_counts.head()

In [ ]:
person_active_counts_summary = (
    person_active_counts.loc[person_active_counts.valid_day == 1, :]
    .groupby(["SEQN"])
    .agg({"worn_minutes": np.mean, "activity_counts": np.mean, "valid_day": "count"})
)
person_active_counts_summary.columns = [
    "daily_worn_minutes_mean",
    "daily_activity_count_sum_mean",
    "n_valid_days",
]
person_active_counts_summary.head()

In [ ]:
person_active_counts_summary.shape

### Save off this `person_active_counts_summary` as the "`fishman_summary`" file

Note that at one point I was generating files that looked like:
```
In [7]: pd.read_parquet('data/NHANES/2003-2004/PAXRAW_C_int_dt_fishman_summary.parquet').head()
Out[7]:
       daily_worn_minutes_mean  daily_activity_count_sum_mean  n_valid_days
SEQN
21005               810.000000                  523275.000000             3
21006               734.500000                  117279.250000             4
21007               911.571429                  361203.714286             7
21008               788.666667                  214938.333333             3
21009               900.428571                  409360.142857             7

In [8]: d = pd.read_parquet('data/NHANES/2003-2004/PAXRAW_C_int_dt_fishman.parquet')

In [9]: d.loc[d.SEQN == 21005, :]
Out[9]:
    SEQN  PAXDAY  worn_sum  PAXINTEN_sum  valid_day  max_intensity_sum  max_intensity_last  out_of_calibration_sum  out_of_calibration_last  unreliable_sum  unreliable_last
0  21005       1       202         32695      False                  0               False                       0                    False               0            False
1  21005       2        76          5380      False                  0               False                       0                    False               0            False
2  21005       3       243        143783      False                  0               False                       0                    False               0            False
3  21005       4       873        851618       True                  0               False                       0                    False               0            False
4  21005       5       203         74453      False                  0               False                       0                    False               0            False
5  21005       6       681        249123       True                  0               False                       0                    False               0            False
6  21005       7       876        469084       True                  0               False                       0                    False               0            False
```

In [ ]:
person_active_counts_summary.to_parquet(
    datadir / f"paxraw_{CHAR_LOOKUP[year].lower()}_fishman_summary.parquet"
)

In [ ]:
alt.Chart(person_active_counts_summary).mark_bar().encode(
    alt.X("daily_worn_minutes_mean:Q", bin=True),
    y="count()",
)

In [ ]:
alt.Chart(person_active_counts_summary).mark_bar().encode(
    alt.X("daily_activity_count_sum_mean:Q", bin=True),
    y="count()",
)

In [ ]:
alt.Chart(person_active_counts_summary).mark_bar(clip=True).encode(
    alt.X(
        "daily_activity_count_sum_mean:Q",
        scale=alt.Scale(domain=[0, 5000000]),
        bin=alt.Bin(maxbins=500),
    ),
    y="count()",
)

In [ ]:
alt.Chart(person_active_counts_summary).mark_bar(clip=True).encode(
    alt.X(
        "daily_activity_count_sum_mean:Q",
        scale=alt.Scale(domain=[0, 1500000]),
        bin=alt.Bin(maxbins=1000),
    ),
    y="count()",
)

In [ ]:
alt.Chart(person_active_counts_summary).mark_bar().encode(
    alt.X("daily_activity_count_sum_mean:Q", scale=alt.Scale(type="log")),
    y="count()",
)

In [ ]:
person_active_counts_summary["daily_activity_count_sum_mean_log"] = np.log10(
    person_active_counts_summary["daily_activity_count_sum_mean"]
)

In [ ]:
alt.Chart(person_active_counts_summary).mark_bar(clip=True).encode(
    alt.X(
        "daily_activity_count_sum_mean_log:Q",
        scale=alt.Scale(),
        bin=alt.Bin(maxbins=100),
    ),
    y="count()",
)

In [ ]:
alt.Chart(person_active_counts_summary).mark_bar().encode(
    alt.X("n_valid_days:O"),
    y="count()",
)

In [ ]:
# clear the crazy data
person_active_counts_summary = person_active_counts_summary.loc[
    person_active_counts_summary.daily_activity_count_sum_mean < 1500000, :
].copy()

In [ ]:
cuts = 3
person_active_counts_summary["activity_tertile"] = pd.qcut(
    person_active_counts_summary.daily_activity_count_sum_mean,
    q=cuts,
    labels=range(1, cuts + 1),
)

In [ ]:
alt.Chart(person_active_counts_summary).mark_bar(clip=True).encode(
    alt.X("activity_tertile:O", scale=alt.Scale(), bin=False),
    y="mean(daily_activity_count_sum_mean)",
).properties(width=400)

## Examine the filters for valid people

In [ ]:
d_people.loc[
    (d_people.zero_steps_with_intensity_sum > 10)
    | (d_people.too_many_steps_sum > 10)
    | (d_people.max_intensity_sum > 10),
    :,
].head(20)

In [ ]:
d_people.loc[
    (d_people.zero_steps_with_intensity_sum > 10)
    | (d_people.too_many_steps_sum > 10)
    | (d_people.max_intensity_sum > 10)
    | (d_people.out_of_calibration_last),
    :,
].head(20)

In [ ]:
d_people.loc[
    (d_people.zero_steps_with_intensity_sum > 10)
    | (d_people.too_many_steps_sum > 10)
    | (d_people.max_intensity_sum > 10)
    | (d_people.out_of_calibration_last)
    | (d_people.unreliable_last),
    :,
].head(20)

In [ ]:
d_unreliable = d_people.loc[
    (d_people.zero_steps_with_intensity_sum > 10)
    | (d_people.too_many_steps_sum > 10)
    | (d_people.max_intensity_sum > 10)
    | (d_people.out_of_calibration_last)
    | (d_people.unreliable_sum > 10)
    | (d_people.steps_filtered_500_sum > 200000),
    :,
]
d_unreliable.head(20)

In [ ]:
alt.Chart(d_unreliable).mark_point().encode(
    alt.X("PAXINTEN_sum", title="Intensity"),
    alt.Y("steps_filtered_500_sum", title="Steps"),
    alt.Color("valid_day", title="N Valid Days"),
)

In [ ]:
alt.Chart(d_reliable).mark_point().encode(
    alt.X("PAXINTEN_sum", title="Intensity"),
    alt.Y("steps_filtered_500_sum", title="Steps"),
    alt.Color("valid_day", title="N Valid Days"),
).properties(width=600, height=600)

# Correlation of steps and intensity

In [ ]:
results = smf.ols("steps_filtered_500_sum ~ PAXINTEN_sum + 0", data=d_reliable).fit()
results.summary()

In [ ]:
alt.Chart(d_reliable).mark_point().encode(
    alt.X("PAXINTEN_sum", title="Intensity"),
    alt.Y("steps_filtered_300_sum", title="Steps"),
    alt.Color("valid_day", title="N Valid Days"),
).properties(width=600, height=600)

In [ ]:
results = smf.ols("steps_filtered_300_sum ~ PAXINTEN_sum + 0", data=d_reliable).fit()
results.summary()

In [ ]:
alt.Chart(d_reliable).mark_point().encode(
    alt.X("steps_filtered_300_sum", title="Steps, 300 threshold"),
    alt.Y("steps_filtered_500_sum", title="Steps, 500 threshold"),
    alt.Color("valid_day", title="N Valid Days"),
).properties(width=600, height=600)